In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
#%pip install pycuda

In [ ]:
import os
import struct
import random
#import pycuda
#import pycuda.autoinit
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge
#from pycuda.tools import make_default_context

In [ ]:
PATH = "drive/MyDrive/data_pfe"
#PATH = "drive/MyDrive/TSP/3A/PFE/data"

## Preprocessing data

In [ ]:
with open(os.path.join(PATH, 'train-images.idx3-ubyte'),'rb') as f:
    magic, size = struct.unpack(">II", f.read(8))
    nrows, ncols = struct.unpack(">II", f.read(8))
    train_images = np.fromfile(f, dtype=np.dtype(np.uint8).newbyteorder('>'))
    train_images = train_images.reshape((size, nrows, ncols))
f.close()

with open(os.path.join(PATH, 'train-labels.idx1-ubyte'),'rb') as f:
    magic, size = struct.unpack(">II", f.read(8))
    train_labels = np.fromfile(f, dtype=np.dtype(np.uint8).newbyteorder('>'))
    train_labels = train_labels.reshape((size,))
f.close()

with open(os.path.join(PATH, 't10k-images.idx3-ubyte'),'rb') as f:
    magic, size = struct.unpack(">II", f.read(8))
    nrows, ncols = struct.unpack(">II", f.read(8))
    test_images = np.fromfile(f, dtype=np.dtype(np.uint8).newbyteorder('>'))
    test_images = test_images.reshape((size, nrows, ncols))
f.close()

with open(os.path.join(PATH, 't10k-labels.idx1-ubyte'),'rb') as f:
    magic, size = struct.unpack(">II", f.read(8))
    test_labels = np.fromfile(f, dtype=np.dtype(np.uint8).newbyteorder('>'))
    test_labels = test_labels.reshape((size,))
f.close()

In [ ]:
print("Shape of the training images set : ", np.shape(train_images))
print("Shape of the training labels set : ", np.shape(train_labels))

In [ ]:
plt.imshow(train_images[25], cmap='Greys')
plt.show()
print("Label of the example image : ", train_labels[25])

In [ ]:
# Normalizing the data

train_images = (train_images - np.mean(train_images)) / np.std(train_images)
test_images = (test_images - np.mean(test_images)) / np.std(test_images)

In [ ]:
def images_to_TS(images, labels) : 

  n_images, img_size, _ = np.shape(images)
  T = n_images * img_size
  U = np.zeros((T, img_size))
  V = np.zeros((T, 10))

  for i in range(n_images) : 
    U[i*img_size : (i+1)*img_size] = images[i]
    for j in range(img_size) : 
      V[i*img_size + j][labels[i]] = 1

  return U, V

U_train, V_train = images_to_TS(train_images, train_labels)
U_test, V_test = images_to_TS(test_images, test_labels)

In [ ]:
# Random shuffle

#idx_train = np.random.permutation(np.shape(U_train)[0])
#U_train, V_train = U_train[idx_train], V_train[idx_train]
#idx_test = np.random.permutation(np.shape(U_test)[0])
#U_test, V_test = U_test[idx_test], V_test[idx_test]


In [ ]:
print("Shape of the training set : \t\t", np.shape(U_train))
print("Shape of the training labels set : \t", np.shape(V_train))
print("Shape of the test set : \t\t", np.shape(U_test))
print("Shape of the test labels set : \t\t", np.shape(V_test))

## Create the ESN model

Notations : 
- **n_inputs** : number of input nodes (same as the image size)
- **n_reservoir** : number of neurons in the reservoir
- **n_outputs** : number of readouts (10 in our case)
- **W_in** : input weights of shape (Nx, Nu)
- **W** : reservoir weights of shape (Nx, Nx)
- **W_back** : return weights of shape (Nx, Ny)
- **W_out** : output weights of shape (Ny, Nx)
- **U** : training set of shape (T, Nu) where T is the length of the times series
- **V** : set of training labels of shape (T, Ny) where the labels are one-hot encoded
- **X** : reservoir states of shape (Nx, T)
- **Y** : labels corresponding to the reservoir states (Ny, T)

In [ ]:
img_size = 28
Nu = img_size     # size of the input signal
Nx = 500           # number of neurons in the reservoir
Ny = 10           # number of readouts

In [ ]:
class ESN_2():

  def param_init_random(self) : 

    self.x_0 = np.zeros(self.Nx)
    self.y_0 = np.zeros(self.Ny)

    self.W_in = np.empty((self.Nx, self.Nu))
    self.W = np.empty((self.Nx, self.Nx))
    self.W_back = np.empty((self.Nx, self.Ny))
    self.W_bias = np.zeros(self.Nx)

    for i in range(self.Nx) : 
      for j in range(self.Nu) : 
        w = np.random.rand()
        if w < 0.2 : 
          self.W_in[i,j] = -1
        elif w >= 0.2 and w < 0.4 : 
          self.W_in[i,j] = 1
        else : 
          self.W_in[i,j] = 0
      for j in range(self.Nx) : 
        if i>=j : 
          self.W[i,j] = np.exp(-i / 240)
      for j in range(self.Ny) : 
        w = np.random.rand()
        if w < 0.2 : 
          self.W_back[i,j] = -1
        elif w >= 0.2 and w < 0.4 : 
          self.W_back[i,j] = 1
        else : 
          self.W_back[i,j] = 0
      self.W_bias[i] = np.random.rand()
    
    return self.W_in, self.W, self.W_back, self.W_bias

  def forward(self, s, alpha) : 
        return (1 - alpha) * np.tanh(s)

  def compute_reservoir_states(self, U, V) : 

    T = np.shape(U)[0]
    X = np.array([self.x_0]).T
    Y = np.array([self.y_0]).T

    for t in range(1, T+1) : 
      s = self.W_in @ U[t-1] + self.W @ X[:, t-1] + self.W_back @ Y[:, t-1] + self.W_bias
      x_t = self.forward(s, self.alpha) 
      x_t.shape = (np.shape(x_t)[0], 1)
      X = np.hstack([X, x_t])
      Y = np.hstack([Y, np.reshape(V[t-1], (np.shape(V[t-1])[0], 1))])

    return X[:, 1:], Y[:, 1:]

  def compute_W_out(self, X, Y, alpha=1.0) : 
    clf = Ridge(alpha=alpha, fit_intercept=False)
    clf.fit(X.T, Y.T)
    r2 = clf.score(X.T, Y.T)
    self.W_out = clf.coef_
    return r2


  def predict(self, U_test, X, Y) : 
    
    n_samples = np.shape(U_test)[0]
    T = np.shape(X)[1]
    
    for i in range(1, n_samples + 1) : 
      s = self.W_in @ U_test[i-1] + self.W @ X[:, T+i-2] + self.W_back @ Y[:, T+i-2] + self.W_bias
      x_i = self.forward(s, self.alpha)
      x_i.shape = (np.shape(x_i)[0], 1)
      X = np.hstack([X, x_i])

      
      # Compute output
      y_i = self.W_out @ x_i
      y_i.shape = (np.shape(y_i)[0], 1)
      Y = np.hstack([Y, y_i])
    
    column_preds = Y[:, T:].T
    preds = []

    n_images = n_samples // self.Nu
    for j in range(n_images) : 
      column_labels = np.argmax(column_preds[j*self.Nu : (j+1)*self.Nu], axis=1)
      label = np.argmax(np.bincount(column_labels))
      preds.append(label)
    
    return np.array(preds)

In [ ]:
def identity(x):
    return x

def tanh(x) : 
  return np.tanh(x)

def forward(self, s, alpha) : 
    return (1 - alpha) * np.tanh(s)

def correct_dimensions(s, targetlength):   # https://github.com/cknd/pyESN/blob/master/pyESN.py
    """checks the dimensionality of some numeric argument s, broadcasts it
       to the specified length if possible.
    Args:
        s: None, scalar or 1D array
        targetlength: expected length of s
    Returns:
        None if s is None, else numpy vector of length targetlength
    """
    if s is not None:
        s = np.array(s)
        if s.ndim == 0:
            s = np.array([s] * targetlength)
        elif s.ndim == 1:
            if not len(s) == targetlength:
                raise ValueError("arg must have length " + str(targetlength))
        else:
            raise ValueError("Invalid argument")
    return s

class ESN() : 

  def __init__(self, n_inputs, n_outputs, 
               n_reservoir=32, alpha=0.99, spectral_radius=1.5, sparsity=0, noise=0.001,
               input_scaling=None, input_shift=None, 
               teacher_forcing=True, teacher_scaling=None, teacher_shift=None,
               feedback_scaling=None, 
               out_activation=identity, inverse_out_activation=identity,
               random_state=None, 
               use_gradient_descent=False, learning_rate=0.01, l2_rate=10e-5) :
    
    # spectral radius = spectral radius of W_rec
    # sparsity = proportion of recurrent weights set to zero
    # noise = noise added to each neuron (regularization)
    # random_state = positive seed
    # input_shift: scalar or vector of length n_inputs to add to each input dimension before feeding it to the network.
    # input_scaling: scalar or vector of length n_inputs to multiply with each input dimension before feeding it to the netw.
    # teacher_forcing: if True, feed the target back into output units
    # teacher_scaling: factor applied to the target signal
    # teacher_shift: additive term applied to the target signal
    # out_activation: output activation function (applied to the readout)
    # inverse_out_activation: inverse of the output activation function

    self.n_inputs = n_inputs
    self.n_outputs = n_outputs
    self.n_reservoir = n_reservoir
     
    self.noise = noise
    self.spectral_radius = spectral_radius
    self.alpha = alpha

    self.random_state = random_state
    self.sparsity = sparsity

    self.teacher_forcing = teacher_forcing
    self.teacher_scaling = teacher_scaling
    self.teacher_shift = teacher_shift
    self.feedback_scaling = feedback_scaling
    self.input_shift = correct_dimensions(input_shift, n_inputs)
    self.input_scaling = correct_dimensions(input_scaling, n_inputs)
    
    self.out_activation = identity
    self.inverse_out_activation = identity 

    if isinstance(random_state, np.random.RandomState):
        self.random_state_ = random_state
    elif random_state:
        try:
            self.random_state_ = np.random.RandomState(random_state)
        except TypeError as e:
            raise Exception("Invalid seed: " + str(e))
    else:
        self.random_state_ = np.random.mtrand._rand
    
    self.init_weights()

    self.use_gradient_descent = use_gradient_descent
    self.learning_rate = learning_rate
    self.l2_rate = l2_rate

  def init_weights(self) : 

    self.x_0 = np.zeros(self.n_inputs)
    self.y_0 = np.zeros(self.n_outputs)

    self.W_in = np.empty((self.n_reservoir, self.n_inputs))
    self.W = np.empty((self.n_reservoir, self.n_reservoir))
    self.W_back = np.empty((self.n_reservoir, self.n_outputs))
    self.W_bias = np.zeros(self.n_reservoir)
  
    # initialize recurrent weights:
    # begin with a random matrix centered around zero:
    W = self.random_state_.rand(self.n_reservoir, self.n_reservoir) - 0.5

    # delete the fraction of connections given by (self.sparsity):
    W[self.random_state_.rand(*W.shape) < self.sparsity] = 0
    
    # compute the spectral radius of these weights:
    radius = np.max(np.abs(np.linalg.eigvals(W)))

    # rescale them to reach the requested spectral radius:
    self.W_rec = W * (self.spectral_radius / radius)
    
    # random input weights:
    self.W_in = self.random_state_.rand(
        self.n_reservoir, self.n_inputs) * 2 - 1
    
    # random feedback (teacher forcing) weights:
    self.W_back = self.random_state_.rand(
        self.n_reservoir, self.n_outputs) * 2 - 1
  
  def one_step(self, state, input, output):

    if self.teacher_forcing :
      preact = np.dot(self.W_rec, state) + np.dot(self.W_in, input) + np.dot(self.W_back, output) 
    else :
      preact = np.dot(self.W_rec, state) + np.dot(self.W_in, input)
    
    return forward(self, preact, self.alpha) + self.noise * (self.random_state_.rand(self.n_reservoir) - 0.5)

  def _scale_inputs(self, inputs):
    """for each input dimension j: multiplies by the j'th entry in the
    input_scaling argument, then adds the j'th entry of the input_shift
    argument."""
    if self.input_scaling is not None:
      inputs = np.dot(inputs, np.diag(self.input_scaling))
    if self.input_shift is not None:
      inputs = inputs + self.input_shift
    return inputs

  def _scale_teacher(self, teacher):
    """multiplies the teacher/target signal by the teacher_scaling argument,
    then adds the teacher_shift argument to it."""
    if self.teacher_scaling is not None:
      teacher = teacher * self.teacher_scaling
    if self.teacher_shift is not None:
      teacher = teacher + self.teacher_shift
    return teacher

  def _unscale_teacher(self, teacher_scaled):
    """inverse operation of the _scale_teacher method."""
    if self.teacher_shift is not None:
      teacher_scaled = teacher_scaled - self.teacher_shift
    if self.teacher_scaling is not None:
      teacher_scaled = teacher_scaled / self.teacher_scaling
    return teacher_scaled

  def fit(self, inputs, outputs, inspect=False):
    if inputs.ndim < 2:
      inputs = np.reshape(inputs, (len(inputs), -1))
    if outputs.ndim < 2:
      outputs = np.reshape(outputs, (len(outputs), -1))
    # transform input and teacher signal:
    inputs_scaled = self._scale_inputs(inputs)
    teachers_scaled = self._scale_teacher(outputs)

    if self.use_gradient_descent : 
      self.W_out = self.random_state_.rand(self.n_outputs, self.n_reservoir+self.n_inputs) * 2 - 1

    states = np.zeros((inputs.shape[0], self.n_reservoir))

    print("computing states ...")
    for n in range(1, inputs.shape[0]):
      states[n, :] = self.one_step(states[n-1, :], inputs_scaled[n-1, :], teachers_scaled[n-1, :])
      if self.use_gradient_descent : 
        extended = np.hstack((states[n,:], inputs_scaled[n,:]))
        self.W_out = self.W_out*(1 - self.learning_rate*self.l2_rate)
        self.W_out += self.learning_rate * np.reshape((teachers_scaled[n,:] - self.W_out@extended), (self.n_outputs, 1)) @ np.reshape(extended, (self.n_reservoir+self.n_inputs, 1)).T

    if not self.use_gradient_descent : 
      # including inputs 
      extended = np.hstack((states, inputs_scaled))

      #calculing W_out 
      print("computing W_out ...")
      self.W_out = np.dot(np.linalg.pinv(extended), 
                            self.inverse_out_activation(teachers_scaled)).T

    self.laststate = states[-1,:]
    self.lastinput = inputs[-1,:]
    self.lastoutput = outputs[-1,:]

    print("Training error ...")
    pred_train = self._unscale_teacher(self.out_activation(
        np.dot( extended, self.W_out.T)))
    print(np.sqrt(np.mean((pred_train - outputs)**2)))
    return pred_train

  def score(self, u, v):
    return np.sqrt(np.mean((u - v)**2))

  def predict(self, inputs, continuation=True):
    # if continuation = True, start the RcNN from the last training state

    n_samples = inputs.shape[0]

    if continuation == True : 
      laststate = self.laststate 
      lastinput = self.lastinput
      lastoutput = self.lastoutput 
    
    else :
      laststate = np.zeros(self.n_reservoir)
      lastinput = np.zeros(self.n_inputs)
      lastoutput = np.zeros(self.n_outputs)

    inputs = np.vstack([lastinput, self._scale_inputs(inputs)])
    states = np.vstack([laststate, np.zeros((n_samples, self.n_reservoir))])
    outputs = np.vstack([lastoutput, np.zeros((n_samples, self.n_outputs))])

    for n in range(n_samples): 
      states[n+1, :] = self.one_step(states[n, :], inputs[n+1, :], outputs[n, :])
      outputs[n+1, :] = self.out_activation(np.dot(self.W_out, np.concatenate([states[n+1, :], inputs[n+1, :]])))

    #return self._unscale_teacher(self.out_activation(outputs[1,:]))

    column_preds = self._unscale_teacher(self.out_activation(outputs[1:, :]))
    preds = []

    n_images = n_samples // self.n_inputs
    for j in range(n_images) : 
      column_labels = np.argmax(column_preds[j*self.n_inputs : (j+1)*self.n_inputs], axis=1)
      label = np.argmax(np.bincount(column_labels))
      preds.append(label)
    
    return np.array(preds)

## Test the ESN model

In [ ]:
%%time
esn = ESN(n_inputs = 28, n_outputs = 10, n_reservoir = 100,
          alpha = 0.9, spectral_radius = 0.25, sparsity = 0.95, noise = 0.001)

esn.fit(U_train, V_train)

In [ ]:
%%time
from sklearn.metrics import accuracy_score

preds = esn.predict(U_test[:5600], continuation=True)
print("Accuracy : ", accuracy_score(test_labels[:200], preds[:200]))

In [ ]:
%%time
esn = ESN(n_inputs = 28, n_outputs = 10, n_reservoir = 400,
          alpha = 0.001, spectral_radius = 1, sparsity = 0.5, noise = 0.001, input_scaling=0.1, 
          input_shift=0.05, teacher_scaling=0.001, teacher_shift=0.1, teacher_forcing=False, feedback_scaling=0.2, 
          out_activation=identity, inverse_out_activation=identity, 
          use_gradient_descent=True)

train_preds = esn.fit(U_train[:60000], V_train[:60000])

In [ ]:
%%time
from sklearn.metrics import accuracy_score

preds = esn.predict(U_test[:10000], continuation=True)
print("Accuracy : ", accuracy_score(test_labels[:300], preds[:300]))

In [ ]:
x = [10000, 20000, 30000, 40000, 50000, 60000, 70000, 80000]
y = [0.64, 0.697, 0.763, 0.777, 0.813, 0.797, 0.787, 0.787]
plt.plot(x, y, color= 'r', marker = '+')
plt.grid()
plt.xlabel('Number of training samples')
plt.ylabel('Test accuracy')

In [ ]:
z = [107, 105, 144, 184, 230, 277, 334, 401]
plt.plot(x, z, color = 'r', marker = '+')
plt.grid()
plt.xlabel('Number of training samples')
plt.ylabel('Training time (s)')

In [ ]:
x = [100, 200, 300, 400, 500, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]
y = [0.3865, 0.447, 0.5055, 0.5615, 0.57, 0.606, 0.661, 0.6895, 0.7155, 0.7105, 0.7375, 0.752, 0.763]
plt.plot(x, y, color = 'r', marker = '+')
plt.grid()
plt.xlabel('Number of reservoir neurons')
plt.ylabel('Test accuracy')

In [ ]:
z = [13.6, 18.7, 25.5, 38.4, 33.1, 46.6, 68, 111, 127, 165, 209, 270, 374]
plt.plot(x, z, color = 'r', marker = '+')
plt.grid()
plt.xlabel('Number of reservoir neurons')
plt.ylabel('Training time (s)')

## Hyperparameter tuning

In [ ]:
class ESNEstimator() : 

  def __init__(self, n_inputs, n_outputs, 
               n_reservoir=32, alpha=0.99, spectral_radius=1.5, sparsity=0, 
               noise=0.001, input_scaling=None, input_shift=None, 
               teacher_forcing=True, teacher_scaling=None, teacher_shift=None,
               feedback_scaling=None, 
               out_activation=identity, inverse_out_activation=identity,
               random_state = None, 
               use_gradient_descent=False, learning_rate=0.01, l2_rate=10e-5) :
    
    # spectral radius = spectral radius of W_rec
    # sparsity = proportion of recurrent weights set to zero
    # noise = noise added to each neuron (regularization)
    # random_state = positive seed
    # input_shift: scalar or vector of length n_inputs to add to each input dimension before feeding it to the network.
    # input_scaling: scalar or vector of length n_inputs to multiply with each input dimension before feeding it to the netw.
    # teacher_forcing: if True, feed the target back into output units
    # teacher_scaling: factor applied to the target signal
    # teacher_shift: additive term applied to the target signal
    # out_activation: output activation function (applied to the readout)
    # inverse_out_activation: inverse of the output activation function
    
    self.n_inputs = n_inputs
    self.n_outputs = n_outputs
    self.n_reservoir = n_reservoir
     
    self.noise = noise
    self.spectral_radius = spectral_radius
    self.alpha = alpha

    self.random_state = random_state
    self.sparsity = sparsity

    self.teacher_forcing = teacher_forcing
    self.teacher_scaling = teacher_scaling
    self.teacher_shift = teacher_shift
    self.feedback_scaling = feedback_scaling
    self.input_shift = input_shift
    self.input_scaling = input_scaling
    
    self.out_activation = identity
    self.inverse_out_activation = identity 

    if isinstance(self.random_state, np.random.RandomState):
        self.random_state_ = self.random_state
    elif random_state:
        try:
            self.random_state_ = np.random.RandomState(self.random_state)
        except TypeError as e:
            raise Exception("Invalid seed: " + str(e))
    else:
        self.random_state_ = np.random.mtrand._rand
    
    self.use_gradient_descent = use_gradient_descent
    self.learning_rate = learning_rate
    self.l2_rate = l2_rate

    #self.init_weights()

  def get_params(self, deep=True):
        # Return the hyperparameters as a dictionary
        return {'n_inputs': self.n_inputs, 'n_outputs': self.n_outputs, 'n_reservoir': self.n_reservoir, 
                'alpha': self.alpha, 'spectral_radius': self.spectral_radius, 'sparsity': self.sparsity, 
                'noise': self.noise, 'input_scaling': self.input_scaling, 
                'input_shift': self.input_shift, 'teacher_forcing': self.teacher_forcing, 
                'teacher_scaling': self.teacher_scaling, 'teacher_shift': self.teacher_shift, 
                'feedback_scaling': self.feedback_scaling, 'out_activation': self.out_activation, 
                'inverse_out_activation': self.inverse_out_activation, 'random_state': self.random_state, 
                'use_gradient_descent': self.use_gradient_descent, 'learning_rate': self.learning_rate, 'l2_rate': self.l2_rate}

  def set_params(self, **params):
        # Set the hyperparameters

        self.n_reservoir = params['n_reservoir']
        self.alpha = params['alpha']
        self.spectral_radius = params['spectral_radius']
        self.sparsity = params['sparsity']
        self.noise = params['noise']
        self.input_scaling = params['input_scaling']
        self.input_shift = params['input_scaling']
        self.teacher_forcing = params['teacher_forcing']
        self.teacher_scaling = params['teacher_scaling']
        self.teacher_shift = params['teacher_shift']
        self.feedback_scaling = params['feedback_scaling']
        self.out_activation = params['out_activation']
        self.inverse_out_activation = params['inverse_out_activation']
        self.random_state = params['random_state']
        self.use_gradient_descent = params['use_gradient_descent']
        self.learning_rate = params['learning_rate']
        self.l2_rate = params['l2_rate']
        return self

  def init_weights(self) : 

    self.x_0 = np.zeros(self.n_inputs)
    self.y_0 = np.zeros(self.n_outputs)

    self.W_in = np.empty((self.n_reservoir, self.n_inputs))
    self.W = np.empty((self.n_reservoir, self.n_reservoir))
    self.W_back = np.empty((self.n_reservoir, self.n_outputs))
    self.W_bias = np.zeros(self.n_reservoir)
  
    # initialize recurrent weights:
    # begin with a random matrix centered around zero:
    W = self.random_state_.rand(self.n_reservoir, self.n_reservoir) - 0.5

    # delete the fraction of connections given by (self.sparsity):
    W[self.random_state_.rand(*W.shape) < self.sparsity] = 0
    
    # compute the spectral radius of these weights:
    radius = np.max(np.abs(np.linalg.eigvals(W)))

    # rescale them to reach the requested spectral radius:
    self.W_rec = W * (self.spectral_radius / radius)
    
    # random input weights:
    self.W_in = self.random_state_.rand(
        self.n_reservoir, self.n_inputs) * 2 - 1
    
    # random feedback (teacher forcing) weights:
    self.W_back = self.random_state_.rand(
        self.n_reservoir, self.n_outputs) * 2 - 1
  
  def one_step(self, state, input, output):

    if self.teacher_forcing :
      preact = np.dot(self.W_rec, state) + np.dot(self.W_in, input) + np.dot(self.W_back, output) 
    
    else :
      preact = np.dot(self.W_rec, state) + np.dot(self.W_in, input)
    
    return forward(self, preact, self.alpha) + self.noise * (self.random_state_.rand(self.n_reservoir) - 0.5)

  def _scale_inputs(self, inputs):
    """for each input dimension j: multiplies by the j'th entry in the
    input_scaling argument, then adds the j'th entry of the input_shift
    argument."""
    if self.input_scaling is not None:
      inputs = np.dot(inputs, np.diag(self.input_scaling))
    if self.input_shift is not None:
      inputs = inputs + self.input_shift
    return inputs

  def _scale_teacher(self, teacher):
    """multiplies the teacher/target signal by the teacher_scaling argument,
    then adds the teacher_shift argument to it."""
    if self.teacher_scaling is not None:
      teacher = teacher * self.teacher_scaling
    if self.teacher_shift is not None:
      teacher = teacher + self.teacher_shift
    return teacher

  def _unscale_teacher(self, teacher_scaled):
    """inverse operation of the _scale_teacher method."""
    if self.teacher_shift is not None:
      teacher_scaled = teacher_scaled - self.teacher_shift
    if self.teacher_scaling is not None:
      teacher_scaled = teacher_scaled / self.teacher_scaling
    return teacher_scaled

  def fit(self, inputs, outputs, inspect=False):

    self.init_weights()
    self.input_shift = correct_dimensions(self.input_shift, self.n_inputs)
    self.input_scaling = correct_dimensions(self.input_scaling, self.n_inputs)

    if inputs.ndim < 2:
      inputs = np.reshape(inputs, (len(inputs), -1))
    if outputs.ndim < 2:
      outputs = np.reshape(outputs, (len(outputs), -1))
    # transform input and teacher signal:
    inputs_scaled = self._scale_inputs(inputs)
    teachers_scaled = self._scale_teacher(outputs)

    if self.use_gradient_descent : 
      self.W_out = self.random_state_.rand(self.n_outputs, self.n_reservoir+self.n_inputs) * 2 - 1

    states = np.zeros((inputs.shape[0], self.n_reservoir))

    for n in range(1, inputs.shape[0]):
      states[n, :] = self.one_step(states[n-1, :], inputs_scaled[n-1, :], teachers_scaled[n-1, :])
      if self.use_gradient_descent : 
        extended = np.hstack((states[n,:], inputs_scaled[n,:]))
        self.W_out = self.W_out*(1 - self.learning_rate*self.l2_rate)
        self.W_out += self.learning_rate * np.reshape((teachers_scaled[n,:] - self.W_out@extended), (self.n_outputs, 1)) @ np.reshape(extended, (self.n_reservoir+self.n_inputs, 1)).T

    if not self.use_gradient_descent : 
      # including inputs 
      extended = np.hstack((states, inputs_scaled))

      #calculing W_out 
      self.W_out = np.dot(np.linalg.pinv(extended), 
                          self.inverse_out_activation(teachers_scaled)).T

    self.laststate = states[-1,:]
    self.lastinput = inputs[-1,:]
    self.lastoutput = outputs[-1,:]

    print("Training error ...")
    pred_train = self._unscale_teacher(self.out_activation(
        np.dot( extended, self.W_out.T)))
    print(np.sqrt(np.mean((pred_train - outputs)**2)))
    return pred_train

  def score(self, u, v):
    return np.sqrt(np.mean((u - v)**2))

  def predict(self, inputs, continuation=True):
    # if continuation = True, start the RcNN from the last training state

    n_samples = inputs.shape[0]

    if continuation == True : 
      laststate = self.laststate 
      lastinput = self.lastinput
      lastoutput = self.lastoutput 
    
    else :
      laststate = np.zeros(self.n_reservoir)
      lastinput = np.zeros(self.n_inputs)
      lastoutput = np.zeros(self.n_outputs)

    inputs = np.vstack([lastinput, self._scale_inputs(inputs)])
    states = np.vstack([laststate, np.zeros((n_samples, self.n_reservoir))])
    outputs = np.vstack([lastoutput, np.zeros((n_samples, self.n_outputs))])

    for n in range(n_samples): 
      states[n+1, :] = self.one_step(states[n, :], inputs[n+1, :], outputs[n, :])
      outputs[n+1, :] = self.out_activation(np.dot(self.W_out, np.concatenate([states[n+1, :], inputs[n+1, :]])))

    return self._unscale_teacher(self.out_activation(outputs[1,:]))

    """column_preds = self._unscale_teacher(self.out_activation(outputs[1:, :]))
    preds = []

    n_images = n_samples // self.n_inputs
    for j in range(n_images) : 
      column_labels = np.argmax(column_preds[j*self.n_inputs : (j+1)*self.n_inputs], axis=1)
      label = np.argmax(np.bincount(column_labels))
      preds.append(label)
    
    return np.array(preds)"""

In [ ]:
# GridSearch
%%time
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

esn = ESN(n_inputs=28, n_outputs=10)
scoring_func = make_scorer(esn.score)

param_grid = {'n_reservoir': [300],       # peut être fixé (300 ou 400)
          'alpha': [0.001],           # peut être fixé
          'spectral_radius': [1], 
          'sparsity': [0.5], 
          'noise': [0.001], 
          'input_scaling': [0.1],        # à affiner
          'input_shift': [0.05], 
          'teacher_forcing': [True], 
          'teacher_scaling': [0.001], 
          'teacher_shift': [0.2], 
          'feedback_scaling': [0.01],     # à affiner
          'out_activation': [identity], 
          'inverse_out_activation': [identity], 
          'random_state': [None], 
          'use_gradient_descent': [True], 
          'learning_rate': [0.00001, 0.0001, 0.001], 
          'l2_rate': [0.0000001, 0.000001, 0.00001]}

gs = GridSearchCV(ESNEstimator(n_inputs=28, n_outputs=10), param_grid=param_grid, scoring=scoring_func)
gs.fit(U_train[:30000], V_train[:30000])

results = pd.DataFrame(gs.cv_results_).drop(columns=["param_inverse_out_activation", "param_out_activation", "params", "split0_test_score", "split1_test_score", "split2_test_score", "split3_test_score", "split4_test_score", "mean_fit_time", "std_fit_time", "mean_score_time", "std_score_time"])
results = results.sort_values(by="rank_test_score")
print(results.head(5))
print("Best score : ", gs.best_score_)
print("Best params : \n", gs.best_params_)

In [ ]:
%%time
best_esn = ESN(n_inputs = 28, n_outputs = 10, 
                        n_reservoir = gs.best_params_['n_reservoir'], 
                        alpha = gs.best_params_['alpha'], 
                        spectral_radius = gs.best_params_['spectral_radius'], 
                        sparsity = gs.best_params_['sparsity'], 
                        noise = gs.best_params_['noise'], 
                        input_scaling = gs.best_params_['input_scaling'], 
                        input_shift = gs.best_params_['input_shift'], 
                        teacher_forcing = gs.best_params_['teacher_forcing'], 
                        teacher_scaling = gs.best_params_['teacher_scaling'], 
                        teacher_shift = gs.best_params_['teacher_shift'], 
                        feedback_scaling = gs.best_params_['feedback_scaling'], 
                        out_activation = gs.best_params_['out_activation'], 
                        inverse_out_activation = gs.best_params_['inverse_out_activation'], 
                        random_state = gs.best_params_['random_state'], 
                        use_gradient_descent = gs.best_params_['use_gradient_descent'], 
                        learning_rate = gs.best_params_['learning_rate'], 
                        l2_rate = gs.best_params_['l2_rate']
)

train_preds = best_esn.fit(U_train[:60000], V_train[:60000])

In [ ]:
%%time
from sklearn.metrics import accuracy_score

preds = best_esn.predict(U_test[:30000], continuation=True)
print("Accuracy : ", accuracy_score(test_labels[:1000], preds[:1000]))

In [ ]:
"""
With params
  {'alpha': 0.8, 'feedback_scaling': 0.9, 'input_scaling': 0.4, 'input_shift': 0.1, 'inverse_out_activation': <function identity at 0x7fc233f234c0>, 
  'n_reservoir': 200, 'noise': 0.001, 'out_activation': <function identity at 0x7fc233f234c0>, 'random_state': None, 'sparsity': 0.5, 'spectral_radius': 0.8, 
  'teacher_forcing': True, 'teacher_scaling': 0.1, 'teacher_shift': None}
we obtain the following accuracy on the test set : 
  0.28

With params
  {'alpha': 0.9, 'feedback_scaling': 0.6, 'input_scaling': 0.6, 'input_shift': 0.1, 'inverse_out_activation': <function identity at 0x7f0b2ef60040>, 
  'n_reservoir': 200, 'noise': 0.001, 'out_activation': <function identity at 0x7f0b2ef60040>, 'random_state': None, 'sparsity': 0.5, 'spectral_radius': 0.8, 
  'teacher_forcing': False, 'teacher_scaling': None, 'teacher_shift': 0.1}
we obtain the following accuracy on the test set : 
  0.343

With params
 {'alpha': 0.5, 'feedback_scaling': 0.6, 'input_scaling': 0.6, 'input_shift': 0.1, 'inverse_out_activation': <function identity at 0x7f0b2ef60040>, 
 'n_reservoir': 200, 'noise': 0.001, 'out_activation': <function identity at 0x7f0b2ef60040>, 'random_state': None, 'sparsity': 0.5, 'spectral_radius': 1, 
 'teacher_forcing': False, 'teacher_scaling': None, 'teacher_shift': 0.1}
we obtain the following accuracy on the test set : 
  0.406

With params
 {'alpha': 0.4, 'feedback_scaling': 0.6, 'input_scaling': 0.6, 'input_shift': 0.1, 'inverse_out_activation': <function identity at 0x7f0b2ef60040>, 
 'n_reservoir': 200, 'noise': 0.001, 'out_activation': <function identity at 0x7f0b2ef60040>, 'random_state': None, 'sparsity': 0.5, 'spectral_radius': 0.8, 
 'teacher_forcing': False, 'teacher_scaling': None, 'teacher_shift': 0.1}
we obtain the following accuracy on the test set : 
  0.428

With params
 {'alpha': 0.3, 'feedback_scaling': 0.6, 'input_scaling': 0.6, 'input_shift': 0.2, 'inverse_out_activation': <function identity at 0x7f0b2ef60040>, 
 'n_reservoir': 200, 'noise': 0.001, 'out_activation': <function identity at 0x7f0b2ef60040>, 'random_state': None, 'sparsity': 0.5, 'spectral_radius': 1, 
 'teacher_forcing': False, 'teacher_scaling': None, 'teacher_shift': 0.4}
we obtain the following accuracy on the test set : 
  0.45
"""

In [ ]:
!pip install scikeras

In [ ]:
#random search 
%%time 

from sklearn.model_selection import RandomizedSearchCV  
from scipy.stats import uniform, norm
from sklearn.metrics import make_scorer

esn = ESNEstimator(n_inputs=28, n_outputs=10)
scoring_func = make_scorer(esn.score)

distrib = dict(n_reservoir = [2500], 
               alpha = norm(0.00001, 0.01), 
               spectral_radius = [0.4, 0.6, 0.8, 1], 
               sparsity = [0.5], 
               noise = [0.001],  
               input_scaling = norm(0.1, 0.01), 
               input_shift = norm(0.05, 0.005), 
               teacher_scaling = norm(0.01, 0.001), 
               teacher_shift = norm(0.2, 0.02), 
               teacher_forcing = [True], 
               random_state = [None], 
               feedback_scaling = norm(0.01, 0.001),
               out_activation = [identity],  
               inverse_out_activation = [identity], 
               use_gradient_descent = [True],
               learning_rate = norm(0.001, 0.0001), 
               l2_rate = norm(0.001, 0.0001))

clf = RandomizedSearchCV(esn, distrib, random_state=0, scoring=scoring_func)

search = clf.fit(U_train[:30000], V_train[:30000])
search.best_params_


In [ ]:
%%time
best_esn = ESN(n_inputs = 28, n_outputs = 10, 
                n_reservoir = search.best_params_['n_reservoir'],
                alpha = search.best_params_['alpha'], 
                spectral_radius = search.best_params_['spectral_radius'], 
                sparsity = search.best_params_['sparsity'], 
                noise = search.best_params_['noise'], 
                input_scaling = search.best_params_['input_scaling'], 
                input_shift = search.best_params_['input_shift'], 
                teacher_forcing = search.best_params_['teacher_forcing'], 
                teacher_scaling = search.best_params_['teacher_scaling'], 
                teacher_shift = search.best_params_['teacher_shift'], 
                feedback_scaling = search.best_params_['feedback_scaling'], 
                out_activation = search.best_params_['out_activation'], 
                inverse_out_activation = search.best_params_['inverse_out_activation'], 
                random_state = search.best_params_['random_state'], 
                use_gradient_descent = search.best_params_['use_gradient_descent'], 
                learning_rate = search.best_params_['learning_rate'], 
                l2_rate = search.best_params_['l2_rate']
)
               
train_preds = best_esn.fit(U_train[:30000], V_train[:30000])

In [ ]:
%%time
from sklearn.metrics import accuracy_score

preds = best_esn.predict(U_test[:60000], continuation=True)
print("Accuracy : ", accuracy_score(test_labels[:1000], preds[:1000]))

In [ ]:
!echo "# RCN_ESN_RLS" >> README.md
!git init
!git add README.md
!git commit -m "first commit"
!git branch -M main
!git remote add origin https://github.com/blt-tsp/RCN_ESN_RLS.git
!git push -u origin main

## ESN avec GPU

In [ ]:
import pycuda.driver as cuda
from pycuda.compiler import SourceModule

def matmul_gpu(a, b) : 

  n, m = np.shape(a)
  m_, p = np.shape(b)
  if m != m_ : 
    raise ValueError("Invalid arguments")
  n = np.int32(n)
  m = np.int32(m)
  p = np.int32(p)

  c = np.zeros((n, p), dtype=np.float32)

  a_gpu = cuda.mem_alloc(a.size * a.dtype.itemsize)
  b_gpu = cuda.mem_alloc(b.size * b.dtype.itemsize)
  c_gpu = cuda.mem_alloc(c.size * c.dtype.itemsize)

  cuda.memcpy_htod(a_gpu, a)
  cuda.memcpy_htod(b_gpu, b)

  mod = SourceModule("""
    __global__ void multiply
    ( int n, int m, int p,
    float *a, float *b, float *c )
    {
    int idx = p*threadIdx.x + threadIdx.y;
    c[idx] = 0.0;
    for(int k=0; k<m; k++)
    c[idx] += a[m*threadIdx.x+k]
    *b[threadIdx.y+k*p];
    }
    """)
  
  func = mod.get_function("multiply")
  func(n, m, p, a_gpu, b_gpu, c_gpu, block=(int(n), int(p), 1), grid=(1, 1), shared=0)
  cuda.memcpy_dtoh(c, c_gpu)
  return c

In [ ]:
class ESN_gpu() : 

  def __init__(self, n_inputs, n_outputs, 
               n_reservoir=32, alpha=0.99, spectral_radius=1.5, sparsity=0, noise=0.001,
               input_scaling=None, input_shift=None, 
               teacher_forcing=True, teacher_scaling=None, teacher_shift=None,
               feedback_scaling=None, 
               out_activation=identity, inverse_out_activation=identity,
               random_state = None) :
    
    # spectral radius = spectral radius of W_rec
    # sparsity = proportion of recurrent weights set to zero
    # noise = noise added to each neuron (regularization)
    # random_state = positive seed
    # input_shift: scalar or vector of length n_inputs to add to each input dimension before feeding it to the network.
    # input_scaling: scalar or vector of length n_inputs to multiply with each input dimension before feeding it to the netw.
    # teacher_forcing: if True, feed the target back into output units
    # teacher_scaling: factor applied to the target signal
    # teacher_shift: additive term applied to the target signal
    # out_activation: output activation function (applied to the readout)
    # inverse_out_activation: inverse of the output activation function

    self.n_inputs = n_inputs
    self.n_outputs = n_outputs
    self.n_reservoir = n_reservoir
     
    self.noise = noise
    self.spectral_radius = spectral_radius
    self.alpha = alpha

    self.random_state = random_state
    self.sparsity = sparsity

    self.teacher_forcing = teacher_forcing
    self.teacher_scaling = teacher_scaling
    self.teacher_shift = teacher_shift
    self.feedback_scaling = feedback_scaling
    self.input_shift = correct_dimensions(input_shift, n_inputs)
    self.input_scaling = correct_dimensions(input_scaling, n_inputs)
    
    self.out_activation = identity
    self.inverse_out_activation = identity 

    if isinstance(random_state, np.random.RandomState):
        self.random_state_ = random_state
    elif random_state:
        try:
            self.random_state_ = np.random.RandomState(random_state)
        except TypeError as e:
            raise Exception("Invalid seed: " + str(e))
    else:
        self.random_state_ = np.random.mtrand._rand
    
    self.init_weights()

  def get_params(self, deep=True):
        # Return the hyperparameters as a dictionary
        return {'n_inputs': self.n_inputs, 'n_outputs': self.n_outputs, 'n_reservoir': self.n_reservoir, 'alpha': self.alpha, 
                'spectral_radius': self.spectral_radius, 'sparsity': self.sparsity, 'noise': self.noise, 
                'input_scaling': self.input_scaling, 'input_shift': self.input_shift, 'teacher_forcing': self.teacher_forcing, 
                'teacher_scaling': self.teacher_scaling, 'teacher_shift': self.teacher_shift, 
                'feedback_scaling': self.feedback_scaling, 'out_activation': self.out_activation, 
                'inverse_out_activation': self.inverse_out_activation, 'random_state': self.random_state}

  def set_params(self, **params):
        # Set the hyperparameters
        self.n_inputs = params['n_inputs']
        self.n_outputs = params['n_outputs']
        self.n_reservoir = params['n_reservoir']
        self.alpha = params['alpha']
        self.spectral_radius = params['spectral_radius']
        self.sparsity = params['sparsity']
        self.noise = params['noise']
        self.input_scaling = params['input_scaling']
        self.input_shift = params['input_scaling']
        self.teacher_forcing = params['teacher_forcing']
        self.teacher_scaling = params['teacher_forcing']
        self.teacher_shift = params['teacher_shift']
        self.feedback_scaling = params['feedback_scaling']
        self.out_activation = params['out_activation']
        self.inverse_out_activation = params['inverse_out_activation']
        self.random_state = params['random_state']
        return self

  def init_weights(self) : 

    self.x_0 = np.zeros(self.n_inputs)
    self.y_0 = np.zeros(self.n_outputs)

    self.W_in = np.empty((self.n_reservoir, self.n_inputs))
    self.W = np.empty((self.n_reservoir, self.n_reservoir))
    self.W_back = np.empty((self.n_reservoir, self.n_outputs))
    self.W_bias = np.zeros(self.n_reservoir)
  
    # initialize recurrent weights:
    # begin with a random matrix centered around zero:
    W = self.random_state_.rand(self.n_reservoir, self.n_reservoir) - 0.5

    # delete the fraction of connections given by (self.sparsity):
    W[self.random_state_.rand(*W.shape) < self.sparsity] = 0
    
    # compute the spectral radius of these weights:
    radius = np.max(np.abs(np.linalg.eigvals(W)))

    # rescale them to reach the requested spectral radius:
    self.W_rec = W * (self.spectral_radius / radius)
    
    # random input weights:
    self.W_in = self.random_state_.rand(
        self.n_reservoir, self.n_inputs) * 2 - 1
    
    # random feedback (teacher forcing) weights:
    self.W_back = self.random_state_.rand(
        self.n_reservoir, self.n_outputs) * 2 - 1
  
  def one_step(self, state, input, output):

    if self.teacher_forcing :
      state.shape = (np.shape(state)[0], 1)
      input.shape = (np.shape(input)[0], 1)
      output.shape = (np.shape(output)[0], 1)
      preact = matmul_gpu(self.W_rec, state) + matmul_gpu(self.W_in, input) + matmul_gpu(self.W_back, output) 
    else :
      state.shape = (np.shape(state)[0], 1)
      input.shape = (np.shape(input)[0], 1)
      preact = matmul_gpu(self.W_rec, state) + matmul_gpu(self.W_in, input)
    preact.shape = (np.shape(preact)[0],)
    
    return forward(self, preact, self.alpha) + self.noise * (self.random_state_.rand(self.n_reservoir) - 0.5)

  def _scale_inputs(self, inputs):
    """for each input dimension j: multiplies by the j'th entry in the
    input_scaling argument, then adds the j'th entry of the input_shift
    argument."""
    if self.input_scaling is not None:
      inputs = np.dot(inputs, np.diag(self.input_scaling))
    if self.input_shift is not None:
      inputs = inputs + self.input_shift
    return inputs

  def _scale_teacher(self, teacher):
    """multiplies the teacher/target signal by the teacher_scaling argument,
    then adds the teacher_shift argument to it."""
    if self.teacher_scaling is not None:
      teacher = teacher * self.teacher_scaling
    if self.teacher_shift is not None:
      teacher = teacher + self.teacher_shift
    return teacher

  def _unscale_teacher(self, teacher_scaled):
    """inverse operation of the _scale_teacher method."""
    if self.teacher_shift is not None:
      teacher_scaled = teacher_scaled - self.teacher_shift
    if self.teacher_scaling is not None:
      teacher_scaled = teacher_scaled / self.teacher_scaling
    return teacher_scaled

  def fit(self, inputs, outputs, inspect=False):
    if inputs.ndim < 2:
      inputs = np.reshape(inputs, (len(inputs), -1))
    if outputs.ndim < 2:
      outputs = np.reshape(outputs, (len(outputs), -1))
    # transform input and teacher signal:
    inputs_scaled = self._scale_inputs(inputs)
    teachers_scaled = self._scale_teacher(outputs)

    states = np.zeros((inputs.shape[0], self.n_reservoir))

    print("computing states ...")
    for n in range(1, inputs.shape[0]):
      states[n, :] = self.one_step(states[n-1, :], inputs_scaled[n-1, :], teachers_scaled[n-1, :])

    # including inputs 
    extended = np.hstack((states, inputs_scaled))

    #calculing W_out 
    print("computing W_out ...")
    print(np.shape(np.linalg.pinv(extended)))
    print("ok")
    self.W_out = matmul_gpu(np.linalg.pinv(extended), 
                        self.inverse_out_activation(teachers_scaled)).T
    print("ok")

    self.laststate = states[-1,:]
    self.lastinput = inputs[-1,:]
    self.lastoutput = outputs[-1,:]

    print("Training error ...")
    pred_train = self._unscale_teacher(self.out_activation(
        matmul_gpu( extended, self.W_out.T)))
    print(np.sqrt(np.mean((pred_train - outputs)**2)))
    return pred_train

  def score(self, u, v):
    return np.sqrt(np.mean((u - v)**2))

  def predict(self, inputs, continuation=True):
    # if continuation = True, start the RcNN from the last training state

    n_samples = inputs.shape[0]

    if continuation == True : 
      laststate = self.laststate 
      lastinput = self.lastinput
      lastoutputt = self.lastoutput 
    
    else :
      laststate = np.zeros(self.n_reservoir)
      lastinput = np.zeros(self.n_inputs)
      lastoutput = np.zeros(self.n_outputs)

    inputs = np.vstack([lastinput, self._scale_inputs(inputs)])
    states = np.vstack([laststate, np.zeros((n_samples, self.n_reservoir))])
    outputs = np.vstack([lastoutput, np.zeros((n_samples, self.n_outputs))])

    for n in range(n_samples): 
      states[n+1, :] = self.one_step(states[n, :], inputs[n+1, :], outputs[n, :])
      outputs[n+1, :] = self.out_activation(matmul_gpu(self.W_out, np.concatenate([states[n+1, :], inputs[n+1, :]])))

    #return self._unscale_teacher(self.out_activation(outputs[1,:]))

    column_preds = self._unscale_teacher(self.out_activation(outputs[1:, :]))
    preds = []

    n_images = n_samples // self.n_inputs
    for j in range(n_images) : 
      column_labels = np.argmax(column_preds[j*self.n_inputs : (j+1)*self.n_inputs], axis=1)
      label = np.argmax(np.bincount(column_labels))
      preds.append(label)
    
    return np.array(preds)

In [ ]:
c = make_default_context()
dev = c.get_device()

In [ ]:
%%time
esn = ESN(n_inputs = 28, n_outputs = 10, n_reservoir = 100,
          alpha = 0.9, spectral_radius = 1.3, sparsity = 0.90, noise = 0.001, input_scaling=0.6)

pred_train = esn.fit(U_train, V_train)

## Test the ESN without the reservoir (only the output layer)

In [ ]:
esn = ESN(img_size, Nx, Ny)

In [ ]:
%%time

esn.compute_W_out(U_train.T, V_train.T, alpha=0.7)

In [ ]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression(fit_intercept=False)
lr.fit(U_train, V_train)
print(lr.score(U_test, V_test))

In [ ]:
from sklearn.linear_model import Lasso

lasso = Lasso(fit_intercept=False)
lasso.fit(U_train, V_train)
print(lasso.score(U_train, V_train))

In [ ]:
from sklearn.linear_model import Ridge

ridge = Ridge(fit_intercept=False)
ridge.fit(U_train, V_train)
print(ridge.score(U_train, V_train))

In [ ]:
# GridSearch
%%time
from sklearn.model_selection import GridSearchCV

clf = Ridge(fit_intercept=False)
params = {
    'alpha' : [0.9,0.95,0.99], 
    'max_iter' : [500, 1000, 5000, 10000],
}

gridsearch = GridSearchCV(clf, param_grid=params)

gridsearch.fit(U_train, V_train)

print(gridsearch.best_score_)
print(gridsearch.best_params_)

## GPU computation

In [ ]:
c = make_default_context()
dev = c.get_device()

In [ ]:
matmul_gpu(2, 3, 4)